In [1]:
import pandas
import researchpy as rp
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp


# Analysis <font color=blue>after </font> combining RF, DT and SVM into 1C

In [2]:
df = pandas.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/skout.xlsx")
df = df[['dataset', 'time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd']", "fc")
df.methods = df.methods.replace("['decision_tree', 'libsvm_svc', 'random_forest']", "3c")
df.methods = df.methods.replace("['libsvm_svc']", "1c")
df.methods = df.methods.replace("['decision_tree']", "1c")
df.methods = df.methods.replace("['random_forest']", "1c")
fsIs_b = ['vowel', 'openml_phpJNxH0q', 'dataset_31_credit-g', 'dataset_40_sonar']
fsIs_m = ['solar-flare_1', 'wine-quality-red', 'dataset_39_ecoli', 'synthetic_control']
fsIl_b = ['AirlinesCodrnaAdult', 'MagicTelescope', 'electricity-normalized', 'phpmPOD5A']
fsIl_m = ['pokerhand-normalized', 'eye_movements', 'avila-tr']
flIs_b = ['audiology', 'arrhythmia', 'AP_Breast_Lung', 'AP_Omentum_Ovary']
flIs_m = ['Amazon', 'umistfacescropped', 'phpGUrE90']
flIl_b = ['gina_agnostic', 'hiva_agnostic', 'phpZrCzJR', 'phprAeXmK']
flIl_m = ['KDDCup99', 'connect-4', 'dataset_60_waveform-5000', 'dataset_186_satimage']
df_binary = df[df.dataset.isin(fsIs_b) | df.dataset.isin(fsIl_b) | df.dataset.isin(flIs_b) | df.dataset.isin(flIl_b)]
df_multi = df[df.dataset.isin(fsIs_m) | df.dataset.isin(fsIl_m) | df.dataset.isin(flIs_m) | df.dataset.isin(flIl_m)]
#df.drop(df[(df.methods=='fc') & ((df.time_budget==30) | (df.time_budget==10))].index, inplace=True)
#df.drop(df[(df.methods=='3c') & ((df.time_budget==60) | (df.time_budget==10))].index, inplace=True)
df.shape

(1350, 4)

In [3]:
d20 = ["AirlinesCodrnaAdult",
                    "Amazon",
                    "AP_Breast_Lung",
                    "arrhythmia",
                    "audiology",
                    "connect-4",
                    "dataset_31_credit-g",
                    "electricity-normalized",
                    "eye_movements",
                    "gina_agnostic",
                    "hiva_agnostic",
                    "KDDCup99",
                    "MagicTelescope",
                    "openml_phpJNxH0q",
                    "phpZrCzJR",
                    "pokerhand-normalized",
                    "solar-flare_1",
                    "umistfacescropped",
                    "vowel",
                    "wine-quality-red"]
df = df[df.dataset.isin(d20)]
df.shape

(900, 4)

In [4]:
rp.summary_cont(df['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,900.0,0.796092,0.203655,0.006789,0.782769,0.809415


In [8]:
rp.summary_cont(df.groupby(['time_budget']))['f1score'].round(3)

,N,Mean,SD,SE,95% Conf.,Interval
time_budget,,,,,,
10,300,0.778,0.229,0.013,0.752,0.804
30,300,0.808,0.190,0.011,0.786,0.829
60,300,0.802,0.189,0.011,0.781,0.823


In [6]:
rp.summary_cont(df.groupby(['methods']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
methods,,,,,,
1c,540,0.770990,0.225375,0.009699,0.751981,0.789999
3c,180,0.829777,0.162033,0.012077,0.806106,0.853449
fc,180,0.837713,0.155934,0.011623,0.814933,0.860494


In [7]:
rp.summary_cont(df.groupby(['time_budget', 'methods']))['f1score'].round(3)#.to_latex(index=True)

N   Mean     SD     SE  95% Conf.  Interval
time_budget methods                                               
10          1c       180  0.744  0.257  0.019      0.707     0.782
            3c        60  0.827  0.168  0.022      0.785     0.870
            fc        60  0.831  0.168  0.022      0.789     0.874
30          1c       180  0.788  0.210  0.016      0.758     0.819
            3c        60  0.832  0.155  0.020      0.793     0.871
            fc        60  0.842  0.150  0.019      0.804     0.880
60          1c       180  0.780  0.204  0.015      0.750     0.810
            3c        60  0.830  0.166  0.021      0.788     0.872
            fc        60  0.840  0.152  0.020      0.801     0.878

In [7]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model = ols('f1score ~ C(time_budget)*C(methods)', df).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")

Overall model F( 8, 1341) =  3.019, p =  0.0023


In [8]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     3.019
Date:                Sun, 03 Nov 2019   Prob (F-statistic):            0.00230
Time:                        17:02:40   Log-Likelihood:                 309.31
No. Observations:                1350   AIC:                            -600.6
Df Residuals:                    1341   BIC:                            -553.8
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                                 0.7707      0.012     65.591      0.000       0.748       0.794
C(time_budget)[T.30]                      0.0290      0.017      1.744      0.081      -0.004       0.062
C(time_budget)[T.60]                      0.0257      0.017      1.549      0.122      -0.007       0.058
C(methods)[T.3c]                          0.0589      0.023      2.506      0.012       0.013       0.105
C(methods)[T.fc]                          0.0640      0.023      2.723      0.007       0.018       0.110
C(time_budget)[T.30]:C(methods)[T.3c]    -0.0174      0.033     -0.522      0.601      -0.083       0.048
C(time_budget)[T.60]:C(methods)[T.3c]    -0.0276      0.033     -0.831      0.406      -0.093       0.038
C(time_budget)[T.30]:C(methods)[T.fc]    -0.0244      0.033     -0.734      0.463      -0.090       0.041
C(time_budget)[T.60]:C(methods)[T.fc]    -0.0128      0.033     -0.385      0.700      -0.078       0.052
==============================================================================
Omnibus:                      442.724   Durbin-Watson:                   0.385
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1280.384
Skew:                          -1.686   Prob(JB):                    9.29e-279
Kurtosis:                       6.375   Cond. No.                         11.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
res

,sum_sq,df,F,PR(>F)
C(time_budget),0.111910,2.0,1.501133,0.223252
C(methods),0.746743,2.0,10.016627,0.000048
C(time_budget):C(methods),0.041620,4.0,0.279137,0.891579
Residual,49.986023,1341.0,NaN,NaN


In [10]:
# Fits the model
model2 = ols('f1score ~ C(time_budget)+ C(methods)', df).fit()

print(f"Overall model F({model2.df_model: .0f},{model2.df_resid: .0f}) = {model2.fvalue: .3f}, p = {model2.f_pvalue: .4f}")

Overall model F( 4, 1345) =  5.771, p =  0.0001


In [11]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     5.771
Date:                Sun, 03 Nov 2019   Prob (F-statistic):           0.000132
Time:                        17:02:40   Log-Likelihood:                 308.75
No. Observations:                1350   AIC:                            -607.5
Df Residuals:                    1345   BIC:                            -581.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.7762      0.010     77.221      0.000       0.756       0.796
C(time_budget)[T.30]     0.0206      0.013      1.605      0.109      -0.005       0.046
C(time_budget)[T.60]     0.0176      0.013      1.373      0.170      -0.008       0.043
C(methods)[T.3c]         0.0439      0.014      3.239      0.001       0.017       0.070
C(methods)[T.fc]         0.0516      0.014      3.808      0.000       0.025       0.078
==============================================================================
Omnibus:                      445.365   Durbin-Watson:                   0.385
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1298.908
Skew:                          -1.692   Prob(JB):                    8.83e-283
Kurtosis:                       6.411   Cond. No.                         3.96
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
# Creates the ANOVA table
res2 = sm.stats.anova_lm(model2, typ= 2)
res2

,sum_sq,df,F,PR(>F)
C(time_budget),0.111910,2.0,1.504358,0.222533
C(methods),0.746743,2.0,10.038147,0.000047
Residual,50.027643,1345.0,NaN,NaN


In [13]:
# Calculating effect size
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(res2)

,sum_sq,mean_sq,df,F,PR(>F),eta_sq,omega_sq
C(time_budget),0.111910,0.055955,2.0,1.504358,0.222533,0.002199,0.000737
C(methods),0.746743,0.373372,2.0,10.038147,0.000047,0.014675,0.013203
Residual,50.027643,0.037195,1345.0,NaN,NaN,NaN,NaN


In [14]:
mc = statsmodels.stats.multicomp.MultiComparison(df['f1score'], df['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    1c     3c   0.0439 0.0035  0.0121 0.0757   True
    1c     fc   0.0516  0.001  0.0198 0.0834   True
    3c     fc   0.0077 0.8781 -0.0313 0.0467  False
---------------------------------------------------


In [15]:
mc = statsmodels.stats.multicomp.MultiComparison(df['f1score'], df['time_budget'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    10     30   0.0206 0.2489 -0.0097  0.051  False
    10     60   0.0176 0.3615 -0.0127  0.048  False
    30     60   -0.003    0.9 -0.0334 0.0274  False
---------------------------------------------------


# Analysis <font color=blue>without </font> combining RF, DT and SVM into 1C

In [16]:
df = pandas.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/skout.xlsx")
df = df[['time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd']", "fc")
df.methods = df.methods.replace("['decision_tree', 'libsvm_svc', 'random_forest']", "3c")
df.methods = df.methods.replace("['libsvm_svc']", "svm")
df.methods = df.methods.replace("['decision_tree']", "DT")
df.methods = df.methods.replace("['random_forest']", "RF")
#df.drop(df[(df.methods=='fc') & ((df.time_budget==30) | (df.time_budget==10))].index, inplace=True)
#df.drop(df[(df.methods=='3c') & ((df.time_budget==60) | (df.time_budget==10))].index, inplace=True)
df.shape

(1350, 3)

In [17]:
rp.summary_cont(df['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,1350.0,0.808018,0.19422,0.005286,0.797648,0.818387


In [18]:
rp.summary_cont(df.groupby(['time_budget']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
time_budget,,,,,,
10,450,0.795257,0.216084,0.010186,0.775292,0.815222
30,450,0.815889,0.182823,0.008618,0.798997,0.832781
60,450,0.812907,0.181568,0.008559,0.796131,0.829683


In [19]:
rp.summary_cont(df.groupby(['methods']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
methods,,,,,,
3c,270,0.832812,0.160097,0.009743,0.813716,0.851909
DT,270,0.793107,0.188630,0.011480,0.770607,0.815607
RF,270,0.824734,0.172256,0.010483,0.804187,0.845282
fc,270,0.840522,0.152361,0.009272,0.822348,0.858696
svm,270,0.748912,0.263406,0.016030,0.717493,0.780332


In [20]:
rp.summary_cont(df.groupby(['time_budget', 'methods']))['f1score']

N      Mean        SD        SE  95% Conf.  Interval
time_budget methods                                                       
10          3c       90  0.829569  0.166076  0.017506   0.795257  0.863880
            DT       90  0.778558  0.208695  0.021998   0.735441  0.821674
            RF       90  0.818641  0.199986  0.021080   0.777323  0.859959
            fc       90  0.834680  0.160454  0.016913   0.801530  0.867830
            svm      90  0.714839  0.297571  0.031367   0.653360  0.776318
30          3c       90  0.841189  0.149080  0.015714   0.810389  0.871989
            DT       90  0.802777  0.180722  0.019050   0.765440  0.840115
            RF       90  0.838081  0.155413  0.016382   0.805972  0.870190
            fc       90  0.839267  0.151426  0.015962   0.807982  0.870552
            svm      90  0.758129  0.247810  0.026121   0.706931  0.809327
60          3c       90  0.827679  0.165997  0.017498   0.793384  0.861975
            DT       90  0.797986  0.176059  0.018558   0.761612  0.834360
            RF       90  0.817481  0.158949  0.016755   0.784642  0.850320
            fc       90  0.847619  0.146290  0.015420   0.817395  0.877843
            svm      90  0.773769  0.240550  0.025356   0.724071  0.823467

In [21]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model = ols('f1score ~ C(time_budget)*C(methods)', df).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")

Overall model F( 14, 1335) =  3.434, p =  0.0000


In [22]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     3.434
Date:                Sun, 03 Nov 2019   Prob (F-statistic):           1.63e-05
Time:                        17:02:41   Log-Likelihood:                 321.14
No. Observations:                1350   AIC:                            -612.3
Df Residuals:                    1335   BIC:                            -534.2
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                  0.8296      0.020     41.029      0.000       0.790       0.869
C(time_budget)[T.30]                       0.0116      0.029      0.406      0.685      -0.044       0.068
C(time_budget)[T.60]                      -0.0019      0.029     -0.066      0.947      -0.058       0.054
C(methods)[T.DT]                          -0.0510      0.029     -1.784      0.075      -0.107       0.005
C(methods)[T.RF]                          -0.0109      0.029     -0.382      0.702      -0.067       0.045
C(methods)[T.fc]                           0.0051      0.029      0.179      0.858      -0.051       0.061
C(methods)[T.svm]                         -0.1147      0.029     -4.012      0.000      -0.171      -0.059
C(time_budget)[T.30]:C(methods)[T.DT]      0.0126      0.040      0.312      0.755      -0.067       0.092
C(time_budget)[T.60]:C(methods)[T.DT]      0.0213      0.040      0.527      0.598      -0.058       0.101
C(time_budget)[T.30]:C(methods)[T.RF]      0.0078      0.040      0.193      0.847      -0.072       0.087
C(time_budget)[T.60]:C(methods)[T.RF]      0.0007      0.040      0.018      0.986      -0.079       0.080
C(time_budget)[T.30]:C(methods)[T.fc]     -0.0070      0.040     -0.174      0.862      -0.086       0.072
C(time_budget)[T.60]:C(methods)[T.fc]      0.0148      0.040      0.367      0.714      -0.064       0.094
C(time_budget)[T.30]:C(methods)[T.svm]     0.0317      0.040      0.783      0.434      -0.048       0.111
C(time_budget)[T.60]:C(methods)[T.svm]     0.0608      0.040      1.504      0.133      -0.019       0.140
==============================================================================
Omnibus:                      406.869   Durbin-Watson:                   0.384
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1065.880
Skew:                          -1.589   Prob(JB):                    3.53e-232
Kurtosis:                       5.974   Cond. No.                         21.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
res

,sum_sq,df,F,PR(>F)
C(time_budget),0.111910,2.0,1.520842,2.189063e-01
C(methods),1.529965,4.0,10.396009,2.713434e-08
C(time_budget):C(methods),0.126929,8.0,0.431237,9.027925e-01
Residual,49.117492,1335.0,NaN,NaN


In [24]:
# Fits the model
model2 = ols('f1score ~ C(time_budget)+ C(methods)', df).fit()

print(f"Overall model F({model2.df_model: .0f},{model2.df_resid: .0f}) = {model2.fvalue: .3f}, p = {model2.f_pvalue: .4f}")

Overall model F( 6, 1343) =  7.463, p =  0.0000


In [25]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     7.463
Date:                Sun, 03 Nov 2019   Prob (F-statistic):           7.02e-08
Time:                        17:02:41   Log-Likelihood:                 319.40
No. Observations:                1350   AIC:                            -624.8
Df Residuals:                    1343   BIC:                            -588.3
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.8201      0.014     59.473      0.000       0.793       0.847
C(time_budget)[T.30]     0.0206      0.013      1.616      0.106      -0.004       0.046
C(time_budget)[T.60]     0.0176      0.013      1.383      0.167      -0.007       0.043
C(methods)[T.DT]        -0.0397      0.016     -2.409      0.016      -0.072      -0.007
C(methods)[T.RF]        -0.0081      0.016     -0.490      0.624      -0.040       0.024
C(methods)[T.fc]         0.0077      0.016      0.468      0.640      -0.025       0.040
C(methods)[T.svm]       -0.0839      0.016     -5.091      0.000      -0.116      -0.052
==============================================================================
Omnibus:                      410.580   Durbin-Watson:                   0.384
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1086.768
Skew:                          -1.599   Prob(JB):                    1.03e-236
Kurtosis:                       6.015   Cond. No.                         6.45
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
# Creates the ANOVA table
res2 = sm.stats.anova_lm(model2, typ= 2)
res2

,sum_sq,df,F,PR(>F)
C(time_budget),0.111910,2.0,1.526012,2.177777e-01
C(methods),1.529965,4.0,10.431351,2.537335e-08
Residual,49.244421,1343.0,NaN,NaN


In [27]:
# Calculating effect size
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(res2)

,sum_sq,mean_sq,df,F,PR(>F),eta_sq,omega_sq
C(time_budget),0.111910,0.055955,2.0,1.526012,2.177777e-01,0.002199,0.000758
C(methods),1.529965,0.382491,4.0,10.431351,2.537335e-08,0.030066,0.027164
Residual,49.244421,0.036667,1343.0,NaN,NaN,NaN,NaN


In [28]:
mc = statsmodels.stats.multicomp.MultiComparison(df['f1score'], df['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
    3c     DT  -0.0397 0.1138 -0.0847  0.0053  False
    3c     RF  -0.0081    0.9 -0.0531   0.037  False
    3c     fc   0.0077    0.9 -0.0373  0.0527  False
    3c    svm  -0.0839  0.001 -0.1289 -0.0389   True
    DT     RF   0.0316  0.308 -0.0134  0.0767  False
    DT     fc   0.0474 0.0333  0.0024  0.0924   True
    DT    svm  -0.0442 0.0574 -0.0892  0.0008  False
    RF     fc   0.0158 0.8618 -0.0292  0.0608  False
    RF    svm  -0.0758  0.001 -0.1209 -0.0308   True
    fc    svm  -0.0916  0.001 -0.1366 -0.0466   True
----------------------------------------------------


# Analysis <font color=blue>after </font> combining RF, DT and SVM into 1C 
# Comparing FC (60 min) with 3C(30)

In [29]:
df = pandas.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/skout.xlsx")
df = df[['time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd']", "fc")
df.methods = df.methods.replace("['decision_tree', 'libsvm_svc', 'random_forest']", "3c")
df.methods = df.methods.replace("['libsvm_svc']", "1c")
df.methods = df.methods.replace("['decision_tree']", "1c")
df.methods = df.methods.replace("['random_forest']", "1c")
df.drop(df[(df.methods=='fc') & ((df.time_budget==30) | (df.time_budget==10))].index, inplace=True)
df.drop(df[(df.methods=='3c') & ((df.time_budget==60) | (df.time_budget==10))].index, inplace=True)
df.drop(df[(df.methods=='1c') & ((df.time_budget==60) | (df.time_budget==10))].index, inplace=True)
df.shape

(450, 3)

In [30]:
rp.summary_cont(df['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,450.0,0.817559,0.182238,0.008591,0.800676,0.834442


In [31]:
rp.summary_cont(df.groupby(['time_budget']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
time_budget,,,,,,
30,360,0.810044,0.189599,0.009993,0.790458,0.829630
60,90,0.847619,0.146290,0.015420,0.817395,0.877843


In [32]:
rp.summary_cont(df.groupby(['methods']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
methods,,,,,,
1c,270,0.799662,0.200472,0.012200,0.775750,0.823575
3c,90,0.841189,0.149080,0.015714,0.810389,0.871989
fc,90,0.847619,0.146290,0.015420,0.817395,0.877843


In [33]:
rp.summary_cont(df.groupby(['time_budget', 'methods']))['f1score']

N      Mean        SD        SE  95% Conf.  Interval
time_budget methods                                                        
30          1c       270  0.799662  0.200472  0.012200   0.775750  0.823575
            3c        90  0.841189  0.149080  0.015714   0.810389  0.871989
60          fc        90  0.847619  0.146290  0.015420   0.817395  0.877843

In [34]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model = ols('f1score ~ C(time_budget)*C(methods)', df).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")

Overall model F( 2, 447) =  3.317, p =  0.0372


In [35]:
model.summary()

C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1755: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\statsmodels\base\model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     3.317
Date:                Sun, 03 Nov 2019   Prob (F-statistic):             0.0372
Time:                        17:02:42   Log-Likelihood:                 131.39
No. Observations:                 450   AIC:                            -256.8
Df Residuals:                     447   BIC:                            -244.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                                 0.7997      0.011     72.474      0.000       0.778       0.821
C(time_budget)[T.60]                      0.0160      0.007      2.173      0.030       0.002       0.030
C(methods)[T.3c]                          0.0415      0.022      1.882      0.061      -0.002       0.085
C(methods)[T.fc]                          0.0160      0.007      2.173      0.030       0.002       0.030
C(time_budget)[T.60]:C(methods)[T.3c]          0          0        nan        nan           0           0
C(time_budget)[T.60]:C(methods)[T.fc]     0.0160      0.007      2.173      0.030       0.002       0.030
==============================================================================
Omnibus:                      149.846   Durbin-Watson:                   0.448
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              406.897
Skew:                          -1.628   Prob(JB):                     4.40e-89
Kurtosis:                       6.331   Cond. No.                          inf
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [36]:
# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
res

C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\statsmodels\base\model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  'rank is %d' % (J, J_), ValueWarning)
C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\statsmodels\base\model.py:1768: RuntimeWarning: invalid value encountered in true_divide
  F /= J
C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\statsmodels\base\model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)
C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\statsmodels\base\model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)


,sum_sq,df,F,PR(>F)
C(time_budget),NaN,1.0,NaN,NaN
C(methods),0.295842,2.0,4.500000,0.034445
C(time_budget):C(methods),0.310481,2.0,4.722663,0.030292
Residual,14.693497,447.0,NaN,NaN


In [37]:
# Fits the model
model2 = ols('f1score ~ C(time_budget)+ C(methods)', df).fit()

print(f"Overall model F({model2.df_model: .0f},{model2.df_resid: .0f}) = {model2.fvalue: .3f}, p = {model2.f_pvalue: .4f}")

Overall model F( 2, 447) =  3.317, p =  0.0372


In [38]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     3.317
Date:                Sun, 03 Nov 2019   Prob (F-statistic):             0.0372
Time:                        17:02:42   Log-Likelihood:                 131.39
No. Observations:                 450   AIC:                            -256.8
Df Residuals:                     447   BIC:                            -244.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.7997      0.011     72.474      0.000       0.778       0.821
C(time_budget)[T.60]     0.0240      0.011      2.173      0.030       0.002       0.046
C(methods)[T.3c]         0.0415      0.022      1.882      0.061      -0.002       0.085
C(methods)[T.fc]         0.0240      0.011      2.173      0.030       0.002       0.046
==============================================================================
Omnibus:                      149.846   Durbin-Watson:                   0.448
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              406.897
Skew:                          -1.628   Prob(JB):                     4.40e-89
Kurtosis:                       6.331   Cond. No.                     2.20e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.07e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [39]:
# Creates the ANOVA table
res2 = sm.stats.anova_lm(model2, typ= 2)
res2

,sum_sq,df,F,PR(>F)
C(time_budget),0.155240,1.0,4.722663,0.030292
C(methods),0.218058,2.0,3.316845,0.037162
Residual,14.693497,447.0,NaN,NaN


In [40]:
# Calculating effect size
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(res2)

,sum_sq,mean_sq,df,F,PR(>F),eta_sq,omega_sq
C(time_budget),0.155240,0.155240,1.0,4.722663,0.030292,0.010303,0.008104
C(methods),0.218058,0.109029,2.0,3.316845,0.037162,0.014473,0.010087
Residual,14.693497,0.032871,447.0,NaN,NaN,NaN,NaN


In [41]:
mc = statsmodels.stats.multicomp.MultiComparison(df['f1score'], df['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    1c     3c   0.0415 0.1452 -0.0104 0.0934  False
    1c     fc    0.048  0.077 -0.0039 0.0999  False
    3c     fc   0.0064    0.9 -0.0571   0.07  False
---------------------------------------------------


# Wilcoxon signed-rank test

In [42]:
df = pandas.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/skout.xlsx")
df = df[['time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd']", "fc")
df.methods = df.methods.replace("['decision_tree', 'libsvm_svc', 'random_forest']", "3c")
df.methods = df.methods.replace("['libsvm_svc']", "svm")
df.methods = df.methods.replace("['decision_tree']", "DT")
df.methods = df.methods.replace("['random_forest']", "RF")
df.shape

(1350, 3)

In [43]:
# Wilcoxon signed-rank test
from numpy.random import seed
from numpy.random import randn
from scipy.stats import wilcoxon
# seed the random number generator
seed(1)
# generate two independent samples
data1 = df.f1score[df.methods=='3c']
data2 = df.f1score[df.methods=='fc']
# compare samples
stat, p = wilcoxon(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Same distribution (fail to reject H0)')
else:
    print('Different distribution (reject H0)')

Statistics=14884.000, p=0.022
Different distribution (reject H0)


In [44]:
df = pandas.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/skout.xlsx")
df = df[['time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd']", "fc")
df.methods = df.methods.replace("['decision_tree', 'libsvm_svc', 'random_forest']", "3c")
df.methods = df.methods.replace("['libsvm_svc']", "svm")
df.methods = df.methods.replace("['decision_tree']", "DT")
df.methods = df.methods.replace("['random_forest']", "RF")
df.shape

(1350, 3)

In [45]:
# Wilcoxon signed-rank test
from numpy.random import seed
from numpy.random import randn
from scipy.stats import wilcoxon
# seed the random number generator
seed(1)
# generate two independent samples
data1 = df.f1score[df.methods=='3c']
data2 = df.f1score[df.methods=='fc']
# compare samples
stat, p = wilcoxon(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Same distribution (fail to reject H0)')
else:
    print('Different distribution (reject H0)')

Statistics=14884.000, p=0.022
Different distribution (reject H0)
